Plan de la tâche - Stakeholders' map

1. Création des types de données
2. Création des tables csv
3. Importation dans neo4j sous forme de noeuds-relations
4. Design des requêtes
5. Possibilité d'interroger la base via un notebook Jupyter


Lien du github : https://github.com/MaximeCapron/foodforneo4j

# 5. Possibilité d'interroger la base

In [25]:
import numpy as np
import pandas as pd
from py2neo import Graph
from py2neo.data import Node, Relationship

In [26]:
graph = Graph("bolt://3.90.65.37:34875", auth=("neo4j", "volumes-capes-retrievals"))

## Initialiser le graph

In [27]:
# Adresse du repo github
adresse = "https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/"

In [28]:
def clear():
    # Commencer par supprimer tout ce qui préexiste pour avoir une feuille propre
    try:
        graph.run("MATCH (n) DETACH DELETE n")
    except:
        pass

In [29]:
def initialiser(adresse):
    
        # Tous les noeuds doivent commencer par un index. Les autres colonnes représentent les caractéristiques, 
        # que l'on enregistre.

    noms_relations = ["CONNAISSANCE","SUPERIEUR","FILIALE","EMPLOYE_DANS"]
    specs_humains = [i for i in pd.read_csv(adresse+"Humains.csv",delimiter=";").columns][1:]
    specs_entreprises = [i for i in pd.read_csv(adresse+"Entreprises.csv",delimiter=";").columns][1:]

        # Toutes les relations doivent commencer par les index des deux bouts de la relation. 
        # Le colonnes 2 à n-1 représentent les caractéristiques, que l'on enregistre.
        # La dernière colonne représente le nom, que l'on connaît déjà.

    specs_relations = [0]*len(noms_relations)
    for i in range(len(specs_relations)):
        specs_relations[i] = [j for j in pd.read_csv(adresse+"Relations"+str(i+1)+".csv",delimiter=";").columns][2:-1]

        # Importer les noeuds 

           # Humains

    code_initialisation = "LOAD CSV WITH HEADERS FROM '"+adresse+"""Humains.csv' AS row FIELDTERMINATOR ';'
    MERGE (p:Personne {id: row.index})
    ON CREATE SET """

    for item in specs_humains:
        code_initialisation += "p." + item + " = row." + item + ", "

    try:
        graph.run(code_initialisation[:-2])
    except:
        pass

           # Entreprises

    code_initialisation = "LOAD CSV WITH HEADERS FROM '"+adresse+"""Entreprises.csv' AS row FIELDTERMINATOR ';'
    MERGE (e:Entreprise {id: row.index})
    ON CREATE SET """

    for item in specs_entreprises:
        code_initialisation += "e." + item + " = row." + item + ", "

    try:
        graph.run(code_initialisation[:-2])
    except:
        pass

           # Relations

    for i in range(len(noms_relations)):

        code_initialisation = "LOAD CSV WITH HEADERS FROM '"+adresse+"Relations"+str(i+1)+""".csv' AS rel FIELDTERMINATOR ';'
        MATCH (p1 {id: rel.index})
        MATCH (p2 {id: rel.index_relation})
        MERGE (p1)-[r:"""+noms_relations[i]+"""]->(p2)  """

        if len(specs_relations[i]) > 0:
            code_initialisation += "ON CREATE SET "
            for item in specs_relations[i]:
                code_initialisation += "r." + item + " = rel." + item + ", "

        try:
            graph.run(code_initialisation[:-2])
        except:
            pass

In [30]:
clear()

In [31]:
initialiser(adresse)

## Ajouter ou supprimer un noeud

2 solutions :
- si vous voulez faire quelque chose de lourd, l'option préférable est probablement de modifier les bases de données, en ajoutant, modifiant ou supprimant des lignes, puis en faisant tourner la cellule ci-dessus.
- si vous voulez simplement ajouter ou supprimer un noeud en vous basant sur ses caractéristiques, c'est faisable avec les fonctions ci-dessous.

Vous souhaitez ajouter un nouveau personnage. Il s'appelle Bob, on lui donne l'identifiant 5 (au hasard), et il est boulanger.

In [145]:
Bob = Node("Person",Nom="Bob",emploi="Boulanger",index=5)

In [146]:
graph.create(Bob)

Supprimons Bob.

In [147]:
graph.delete(Bob)

In [148]:
# On peut faire ça, mais ça ne présente pas d'avantage à mon avis.
# (ce sera nécessaire de donner des noms simples aux noeuds pour les relations).
créer_noeud = lambda sorte, **arguments : graph.create(Node(sorte,**arguments))
# créer_noeud("Monsieur",name="Bob",emploi="Boulanger")

## Ajouter ou supprimer une relation

Supposons que l'on a deux noeuds, Bob et Joe, et que l'on veut créer une relation entre eux deux. Précisions une caractéristique pour chacun d'entre deux et la nature de la relation, et allons-y !

In [10]:
Bob = Node("Person",name="Bob",emploi="Boulanger",index=5)
Joe = Node("Person",name="Joe",emploi="Pâtissier",index=5)

In [11]:
graph.create(Relationship(Bob,"AMI",Joe))

In [12]:
graph.delete(Relationship(Bob,"AMI",Joe))

## Afficher des informations utiles

### Qui est mobilisé sur un projet ? Quelles sont leurs opinions ?

Cette fonction affiche la liste des stakeholders sur un dossier, et leur position : "Avocat", "Neutre" ou "A convaincre".

Paramètres : 
- (obligatoire) nom du projet
- (optionnel) position – Ne rien inscrire si vous voulez toutes les positions, sinon inscrire :
    - 1 pour trouver les avocats, 
    - 0 pour les neutres, 
    - -1 pour les stakeholders à convaincre

In [230]:
def positions_projet(nom_du_projet,position="toutes"):
    if(position=="toutes"):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE exists(p.Position_"""+nom_du_projet+""")
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position_"""+nom_du_projet+" AS position ORDER BY position"
    elif(position==1):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE p.Position_"""+nom_du_projet+""" = "Avocat"
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position_"""+nom_du_projet+" AS position ORDER BY rôle"
    elif(position==0):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE p.Position_"""+nom_du_projet+""" = "Neutre"
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position_"""+nom_du_projet+" AS position ORDER BY rôle"
    elif(position==-1):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE p.Position_"""+nom_du_projet+""" = "A convaincre"
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position_"""+nom_du_projet+" AS position ORDER BY rôle"
    else:
        raise "Position inconnue"
    return graph.run(query).to_data_frame()

Certains détracteurs subsistent : à contacter !

### Trouver la distance entre deux personnes

#### En fonction de leurs positions et de leurs entreprises

In [210]:
def distance_pos(A,B):
    query = """
    MATCH (a:Personne)-[:EMPLOYE_DANS {position:'"""+A[0]+"""'}]-(:Entreprise {Titre:'"""+A[1]+"""'})
    MATCH (b:Personne)-[:EMPLOYE_DANS {position:'"""+B[0]+"""'}]-(:Entreprise {Titre:'"""+B[1]+"""'})
    MATCH p = shortestPath((a)-[*]-(b))
    RETURN a.Nom,b.Nom,length(p) AS distance"""
    return graph.run(query).to_data_frame()

In [211]:
A = ["CEO","ECorp"]
B = ["CFO","OBS"]

In [212]:
distance_pos(A,B)

,a.Nom,b.Nom,distance
0,Joséphine Michel,Lucien Martin,2


#### En fonction de leurs noms

In [213]:
def distance_noms(nomA,nomB):
    query = "MATCH p = shortestPath((a:Personne {Nom:'"+nomA+"'})-[*]-(b:Personne {Nom:'"+nomB+"'})) RETURN a.Nom, b.Nom, length(p) AS distance"
    return graph.run(query).to_data_frame()

In [214]:
nomA = "Joseph Leclerc"
nomB = "Auguste Petit"

In [215]:
distance_noms(nomA,nomB)

,a.Nom,b.Nom,distance
0,Joseph Leclerc,Auguste Petit,2


Une seule personne se trouve entre les deux : peut-on lui demander d'intercéder en notre faveur ?

### Quels partenaires n'ont aucune relation directe avec OBS (ne sont pas "surveillées") ?

In [241]:
def a_connaitre():
    query = """
    MATCH (a:Entreprise)-[:EMPLOYE_DANS]-(p:Personne)
    WHERE NOT a.Titre = "OBS"
    WITH p
    MATCH (c:Entreprise {Titre:"OBS"})
    MATCH short=shortestpath((c)-[*]-(p))
    WITH p,length(short) AS len
    WHERE len > 2
    RETURN p.Nom AS nom
    """
    return graph.run(query).to_data_frame()

In [242]:
a_connaitre()

,nom
0,Pierre Leclerc
1,Marthe Guerin
2,Henriette Bonnet
3,Maurice Duval
4,Renée Morin
5,Jules Joly
6,Alice Fournier
7,Anne Gautier
8,Maria Payet
9,René Guillaume


Nous avons une liste de 12 personnes potentielles à contacter !

In [205]:
connue = lambda x : not x in a_connaitre()["nom"].values

In [256]:
def bilan_de_situation(cas):
    print("Voilà les personnes impliquées sur le cas",cas,":\n")
    A = positions_projet(cas)
    A["connue"] = A["nom"].apply(connue)
    return A

In [260]:
bilan_de_situation("D")

Voilà les personnes impliquées sur le cas D :

                   nom entreprise        rôle      position  connue
0        Joseph Thomas      DCorp   Technique  A convaincre    True
1         Anne Gautier      DCorp   Technique  A convaincre   False
2     Juliette Lefevre      CCorp  Commercial        Avocat    True
3        Robert Renard      CCorp         CFO        Avocat    True
4          Jules David      DCorp  Commercial        Avocat    True
5     Lucien Guillaume      DCorp         CFO        Avocat    True
6        Georges Morin      DCorp  Commercial        Avocat    True
7        Hélène Martin      DCorp  Commercial        Avocat    True
8         Emile Moreau      DCorp  Commercial        Avocat    True
9     François Mathieu      CCorp  Commercial        Neutre    True
10          Jules Joly      CCorp   Technique        Neutre   False
11    Joséphine Thomas      DCorp         CEO        Neutre    True
12      Alice Fournier      DCorp   Technique        Neutre   False
1

[I'm a useless link](file:///Users/maxime.capron/Desktop/Pocket%20Result/Graphs/stakeholders'%20map/neovis_graph/Stakeholders.html)